In [1]:
import pandas as pd
from string import punctuation

In [2]:
punctuation += ' '

In [3]:
filename = r"C:\Users\Иннокентий\Downloads\dev.tsv"

In [4]:
sep='>'

In [5]:
df = pd.read_csv(filename, sep='\t', encoding='utf-8', names=['text', 'with_sep'])

In [6]:
df

,text,with_sep
0,Chi uxeʼ ri mexa kʼo wi ri chʼo .,Chi u>xeʼ ri mexa kʼo wi ri chʼo .
1,Na xinnabʼej ta chutasik qakʼolibʼal .,Na x>in>nabʼej ta ch>u>tas>ik qa>kʼolibʼal .
2,Xubʼij che : - jas xabʼano Tintin ?,X>u>bʼij ch>e : - jas x>a>bʼan>o Tintin ?
3,Chanim in kʼo pa mokokil .,Chanim in kʼo pa mokokil .
4,Xalax pa ri Kaweq .,X>alax pa ri Kaweq .
5,"Xpax ri ujukubʼ , ri kej .","X>pax ri u>jukubʼ , ri kej ."
6,Je la leʼ ri nupwiʼ .,Je la leʼ ri nu>pwiʼ .
7,Kʼo pa ja tijobʼal ri wikaqʼ .,Kʼo pa ja tijobʼal ri w>ikaqʼ .
8,Xinalax pa jun qʼij ubʼiʼ wajxaqibʼ Noʼj .,X>in>alax pa jun qʼij u>bʼiʼ wajxaqibʼ Noʼj .
9,Aʼre xetoʼw alaq .,Aʼre x>e>toʼ>w alaq .


In [7]:
kiche_anbur = "a [a], ä [ə], e [ɛ], i [i], o [o], u [u], m [m], n [n], nh [ŋ], p [p], bʼ [ɓ], t [t], tʼ [tʼ], k [k], kʼ [kʼ], q [q], qʼ [qʼ], tz [ʦ], tzʼ [ʦ’], ch [ʧ], chʼ [ʧ’], s [s], x [ʃ], j [χ], h [h], r [r], l [l], y [j], w [w], ʼ [ʔ]"
alphabet = []
for letter in kiche_anbur.split(","):
    letter = letter.split()[0]
    alphabet.append(letter)
    alphabet.append(letter[0].upper() + letter[1:])

In [8]:
print(alphabet)

['a', 'A', 'ä', 'Ä', 'e', 'E', 'i', 'I', 'o', 'O', 'u', 'U', 'm', 'M', 'n', 'N', 'nh', 'Nh', 'p', 'P', 'bʼ', 'Bʼ', 't', 'T', 'tʼ', 'Tʼ', 'k', 'K', 'kʼ', 'Kʼ', 'q', 'Q', 'qʼ', 'Qʼ', 'tz', 'Tz', 'tzʼ', 'Tzʼ', 'ch', 'Ch', 'chʼ', 'Chʼ', 's', 'S', 'x', 'X', 'j', 'J', 'h', 'H', 'r', 'R', 'l', 'L', 'y', 'Y', 'w', 'W', 'ʼ', 'ʼ']


In [9]:
alph_to_int = dict(zip(alphabet, range(len(alphabet))))
int_to_alph = dict(zip(range(len(alphabet)), alphabet))

In [10]:
def window(text, size):
    tokens = []
    i = 0
    while i < (len(text)-size+1):
        try:
            current = ''.join(text[i:i+size])
        except:
            current = 'not_in_alphabet'
        if current in alphabet:
            tokens.append(alph_to_int[current])
            i += size
        else:
            tokens.append(text[i])
            i += 1
    return tokens

In [11]:
def tokenize(text):
    tokens = list(text + 'ггг')
    for i in range(3, 0, -1):
        tokens = window(tokens, i)
    tokens_2 = []
    for token in tokens:
        if token in int_to_alph:
            token = int_to_alph[token]
        tokens_2.append(token)
    return tokens_2

In [12]:
def tokenize_simple(text):
    return(list(text))

In [13]:
#sep — c разделителем, not — без разделителя
#b, m и s значат beginning, middle и end, а s — single
def to_labels(tokens):
    border_tokens = sep + punctuation
    labels = []
    tokens = [sep] + tokens + [sep]
    for i in range(1, len(tokens)-1):
        current = tokens[i]
        previous = tokens[i-1]
        next_ = tokens[i+1]
        if current != sep:
            if current not in punctuation:
                if previous in border_tokens:
                    if next_ in border_tokens:
                        labels.append('s-morph') #labels.append('sep')
                    else:
                        labels.append('b-morph') #labels.append('not')
                elif next_ in border_tokens:
                    labels.append('e-morph') #labels.append('sep')
                else:
                    labels.append('m-morph') #labels.append('not')
            else:
                labels.append(current) #labels.append('punct')
    return labels

In [14]:
df['tokens_with_sep'] = df['with_sep'].apply(tokenize)

In [15]:
df['tokens'] = df['tokens_with_sep'].apply(lambda array: list(filter(lambda x: x!=sep, array)))

In [16]:
df['tokens_honest'] = df['text'].apply(tokenize)

In [17]:
df['tokens_simple'] = df['text'].apply(tokenize_simple)

In [18]:
df['tokens_simple_with_sep'] = df['with_sep'].apply(tokenize_simple)

In [19]:
df['labels'] = df['tokens_simple_with_sep'].apply(to_labels)

In [20]:
df

,text,with_sep,tokens_with_sep,tokens,tokens_honest,tokens_simple,tokens_simple_with_sep,labels
0,Chi uxeʼ ri mexa kʼo wi ri chʼo .,Chi u>xeʼ ri mexa kʼo wi ri chʼo .,"[Ch, i, , u, >, x, e, ʼ, , r, i, , m, e, x,...","[Ch, i, , u, x, e, ʼ, , r, i, , m, e, x, a,...","[Ch, i, , u, x, e, ʼ, , r, i, , m, e, x, a,...","[C, h, i, , u, x, e, ʼ, , r, i, , m, e, x, ...","[C, h, i, , u, >, x, e, ʼ, , r, i, , m, e, ...","[not, not, sep, punct, sep, not, not, sep, pun..."
1,Na xinnabʼej ta chutasik qakʼolibʼal .,Na x>in>nabʼej ta ch>u>tas>ik qa>kʼolibʼal .,"[N, a, , x, >, i, n, >, n, a, bʼ, e, j, , t,...","[N, a, , x, i, n, n, a, bʼ, e, j, , t, a, ,...","[N, a, , x, i, n, n, a, bʼ, e, j, , t, a, ,...","[N, a, , x, i, n, n, a, b, ʼ, e, j, , t, a, ...","[N, a, , x, >, i, n, >, n, a, b, ʼ, e, j, , ...","[not, sep, punct, sep, not, sep, not, not, not..."
2,Xubʼij che : - jas xabʼano Tintin ?,X>u>bʼij ch>e : - jas x>a>bʼan>o Tintin ?,"[X, >, u, >, bʼ, i, j, , ch, >, e, , :, , -...","[X, u, bʼ, i, j, , ch, e, , :, , -, , j, a...","[X, u, bʼ, i, j, , ch, e, , :, , -, , j, a...","[X, u, b, ʼ, i, j, , c, h, e, , :, , -, , ...","[X, >, u, >, b, ʼ, i, j, , c, h, >, e, , :, ...","[sep, sep, not, not, not, sep, punct, not, sep..."
3,Chanim in kʼo pa mokokil .,Chanim in kʼo pa mokokil .,"[Ch, a, n, i, m, , i, n, , kʼ, o, , p, a, ...","[Ch, a, n, i, m, , i, n, , kʼ, o, , p, a, ...","[Ch, a, n, i, m, , i, n, , kʼ, o, , p, a, ...","[C, h, a, n, i, m, , i, n, , k, ʼ, o, , p, ...","[C, h, a, n, i, m, , i, n, , k, ʼ, o, , p, ...","[not, not, not, not, not, sep, punct, not, sep..."
4,Xalax pa ri Kaweq .,X>alax pa ri Kaweq .,"[X, >, a, l, a, x, , p, a, , r, i, , K, a, ...","[X, a, l, a, x, , p, a, , r, i, , K, a, w, ...","[X, a, l, a, x, , p, a, , r, i, , K, a, w, ...","[X, a, l, a, x, , p, a, , r, i, , K, a, w, ...","[X, >, a, l, a, x, , p, a, , r, i, , K, a, ...","[sep, not, not, not, sep, punct, not, sep, pun..."
5,"Xpax ri ujukubʼ , ri kej .","X>pax ri u>jukubʼ , ri kej .","[X, >, p, a, x, , r, i, , u, >, j, u, k, u, ...","[X, p, a, x, , r, i, , u, j, u, k, u, bʼ, ,...","[X, p, a, x, , r, i, , u, j, u, k, u, bʼ, ,...","[X, p, a, x, , r, i, , u, j, u, k, u, b, ʼ, ...","[X, >, p, a, x, , r, i, , u, >, j, u, k, u, ...","[sep, not, not, sep, punct, not, sep, punct, s..."
6,Je la leʼ ri nupwiʼ .,Je la leʼ ri nu>pwiʼ .,"[J, e, , l, a, , l, e, ʼ, , r, i, , n, u, ...","[J, e, , l, a, , l, e, ʼ, , r, i, , n, u, ...","[J, e, , l, a, , l, e, ʼ, , r, i, , n, u, ...","[J, e, , l, a, , l, e, ʼ, , r, i, , n, u, ...","[J, e, , l, a, , l, e, ʼ, , r, i, , n, u, ...","[not, sep, punct, not, sep, punct, not, not, s..."
7,Kʼo pa ja tijobʼal ri wikaqʼ .,Kʼo pa ja tijobʼal ri w>ikaqʼ .,"[Kʼ, o, , p, a, , j, a, , t, i, j, o, bʼ, a...","[Kʼ, o, , p, a, , j, a, , t, i, j, o, bʼ, a...","[Kʼ, o, , p, a, , j, a, , t, i, j, o, bʼ, a...","[K, ʼ, o, , p, a, , j, a, , t, i, j, o, b, ...","[K, ʼ, o, , p, a, , j, a, , t, i, j, o, b, ...","[not, not, sep, punct, not, sep, punct, not, s..."
8,Xinalax pa jun qʼij ubʼiʼ wajxaqibʼ Noʼj .,X>in>alax pa jun qʼij u>bʼiʼ wajxaqibʼ Noʼj .,"[X, >, i, n, >, a, l, a, x, , p, a, , j, u, ...","[X, i, n, a, l, a, x, , p, a, , j, u, n, , ...","[X, i, n, a, l, a, x, , p, a, , j, u, n, , ...","[X, i, n, a, l, a, x, , p, a, , j, u, n, , ...","[X, >, i, n, >, a, l, a, x, , p, a, , j, u, ...","[sep, not, sep, not, not, not, sep, punct, not..."
9,Aʼre xetoʼw alaq .,Aʼre x>e>toʼ>w alaq .,"[A, ʼ, r, e, , x, >, e, >, t, o, ʼ, >, w, , ...","[A, ʼ, r, e, , x, e, t, o, ʼ, w, , a, l, a, ...","[A, ʼ, r, e, , x, e, t, o, ʼ, w, , a, l, a, ...","[A, ʼ, r, e, , x, e, t, o, ʼ, w, , a, l, a, ...","[A, ʼ, r, e, , x, >, e, >, t, o, ʼ, >, w, , ...","[not, not, not, sep, punct, sep, sep, not, not..."


In [21]:
import pickle

In [22]:
filename = filename.split('\\')[-1]
filename = filename.split('.')[0]

In [23]:
with open (filename + '.pickle', 'bw') as f:
    pickle.dump(df, f)